# 11. Training Deep Neural Networks

After introducing relatively shallow nets, let's move on to deeper DNNs (layers >= 10; neurons/layer: X00, X000; connections: X0,000). 

Here are some problems we may encounter along the way，and some techniques we may try out to solve them:

1. Vanishing / Exploding gradients problem making lower layers very hard to train 
                > Initialization 
2. Not enough data / too costly to label 
                > Transfer learning and unsupervised pretraining
3. Painfully slow training 
                > Optimizers to the rescue!
4. Serious overfitting risk for millions params models, especially if there are not enough training instances or if they are too noisy 
                > Good ol' (and new) regularization techniques

### 1. Vanishing / Exploding Gradients Problems

As we know from our previous chapter, Gradient Descent goes from output > input layer propagating the error gradient along the way. Once it has computed the gradient of the cost function for each param of the network, it uses these gradients to update each parameter with a Gradient Descent step.

**Vanishing** gradients gets smaller and smaller, leaving lower layers weights virtually unchanged (no convergence to good solution).  
**Exploding** gradients gets bigger and bigger, making lower layers weights extremely large (divergence).